### To get API key: https://developer.nytimes.com/get-started

In [1]:
from pynytimes import NYTAPI
import datetime
import pandas as pd

import numpy as np

In [2]:
def get_news_specific_stock(start_date: datetime.datetime, end_date: datetime.datetime, nyt=None, k=100):
    """
    get top k most relevent finance news headings on each day from NY times
    """
    res = []
    try:
        #if not nyt:
        #    nyt = NYTAPI(api_key, parse_dates=True)

        articles = nyt.article_search(
            query = "Tesla", 
            results = k,
            dates = {
                "begin": start_date,
                "end": end_date
            },
            options = {
                "sort": "relevance", 
                "sources": [
                    "New York Times",
                    "AP",
                    "Reuters",
                    "International Herald Tribune"
                ],
                # Only get information from these news desk
                "news_desk": [
                    "Business",
                    "Financial",
                    "Technology"
                ],
                "body": [
                    "Tesla",
                ],
            }
        )
        
        for i in range(len(articles)):
            timestamp = articles[i]['pub_date']
            article_url = articles[i]['web_url']
            lead_paragraph = articles[i]['lead_paragraph']
            abstract = articles[i]['abstract']
            #print(articles[i])
            res.append({'timestamp': timestamp,
                        'article_url': article_url,
                        'lead_paragraph': lead_paragraph,
                        'abstract': abstract,
                        })
        
    except Exception as e:
        res.append({})
    
    df = pd.DataFrame(data=res)

    return df

### Change api key, start and end date

In [ ]:
api_key = ""

start_date = datetime.datetime(2024, 1, 1)
end_date = datetime.datetime(2024, 1, 31)

### Get articles

In [ ]:
nyt = NYTAPI(api_key, parse_dates=True)
#k_results =int((end_date - start_date).days * 1.2)  # How many results to pull for. (x1.2 as an estimate for the max articles that exist.)

article_df = get_news_specific_stock(start_date, end_date, nyt) #k=k_results

d:\Programming\sentiment-analysis-volatility-forecasting\venv\lib\site-packages\pynytimes\helpers\article_search.py:33: UserWarning: Asking for a lot of results, because of rate limits it can take a while.
  warnings.warn(


In [10]:
for i, row in article_df.iterrows():
    print(i)
    print(row['lead_paragraph'],'\n', row['abstract'])

0
Elon Musk and Tesla shareholders are at a crossroads. 
 Shares in Elon Musk’s electric vehicle maker fell sharply after the company delivered lackluster quarterly results and declined to give full-year guidance.
1
A Delaware judge’s decision to void the pay package that helped make Elon Musk the world’s richest person leaves Tesla’s board with some difficult decisions to make. 
 The company’s board will now decide whether to appeal the decision, change where Tesla is incorporated or negotiate a new pay package.
2
Tesla’s profit from sales of electric cars slumped in the last three months of last year because of price cuts intended to thwart increasingly intense competition, the company said on Wednesday as it warned of a tough year ahead. 
 A tax benefit boosted quarterly earnings, but that masked a decline in earnings from carmaking caused by intensifying competition.
3
Elon Musk, the chief executive of Tesla, suffered a stunning rebuke Tuesday when a Delaware judge voided the pay p

#### Check if the news falls before or after market closes on each day. If it's after close, then the news should be categorised as a next day news.

In [11]:
threshold_time = pd.Timestamp('16:00:00').time()

article_df['adjusted_date'] = article_df['timestamp'].apply(lambda x: x.date() if x.time() < threshold_time else (x + pd.Timedelta(days=1)).date())
article_df.head()

,timestamp,article_url,lead_paragraph,abstract,adjusted_date
0,2024-01-25 13:04:37+00:00,https://www.nytimes.com/2024/01/25/business/de...,Elon Musk and Tesla shareholders are at a cros...,Shares in Elon Musk’s electric vehicle maker f...,2024-01-25
1,2024-01-31 21:25:41+00:00,https://www.nytimes.com/2024/01/31/business/te...,A Delaware judge’s decision to void the pay pa...,The company’s board will now decide whether to...,2024-02-01
2,2024-01-24 21:21:45+00:00,https://www.nytimes.com/2024/01/24/business/te...,Tesla’s profit from sales of electric cars slu...,"A tax benefit boosted quarterly earnings, but ...",2024-01-25
3,2024-01-30 22:12:10+00:00,https://www.nytimes.com/2024/01/30/business/el...,"Elon Musk, the chief executive of Tesla, suffe...","Shareholders had sued, arguing that Mr. Musk’s...",2024-01-31
4,2024-01-16 14:37:28+00:00,https://www.nytimes.com/2024/01/16/business/te...,"Elon Musk, the chief executive of Tesla, stunn...","Elon Musk, the electric car company’s chief ex...",2024-01-16


### Save file 

In [ ]:
filename = "../data/nyt_tesla.csv"
article_df.to_csv(filename, index=False)